RoPE（Rotary Position Embedding，旋转位置编码）是 Transformer 里用来在注意力计算中注入位置信息的一种方法（GPT-NeoX、LLaMA 都在用）。
核心思想：

•	不直接给 token embedding 加位置向量（像传统的正余弦），而是通过旋转 Query/Key 的二维子空间来注入位置信息。

•	旋转角度由 token 的位置 pos 和频率矩阵 θ 决定。

公式：
\begin{equation}
\text{RoPE}(x, m) =
\begin{bmatrix}
x_{\text{even}}\cos \theta_m - x_{\text{odd}}\sin \theta_m,
x_{\text{even}}\sin \theta_m + x_{\text{odd}}\cos \theta_m
\end{bmatrix}
\end{equation}
其中 $\theta_m = m / 10000^{2i/d}$ 是按频率构造的角度矩阵。

**结论：ROPE是“通过绝对位置编码的方式实现相对位置编码”**

$$
\begin{bmatrix} 
x_{2k-1}^\text{RoPE}(p) \\ x_{2k}^\text{RoPE}(p) 
\end{bmatrix}
=
\begin{bmatrix} 
\cos(p\theta_k) & -\sin(p\theta_k) \\ 
\sin(p\theta_k) & \cos(p\theta_k) 
\end{bmatrix}
\begin{bmatrix} 
x_{2k-1} \\ x_{2k} 
\end{bmatrix}
$$

$\theta_k=\frac{p}{100000^{2k/d}}$

In [ ]:
import torch

inputs = torch.randn(10, 512, 768)

def get_rope_freqs(S, D):
    theta = 10000 **(2 * torch.arange(0, D, 2) / D)
    pos = torch.arange(S)
    freqs = torch.einsum('i,j->ij', pos, theta)
    sin = torch.sin(freqs).repeat_interleave(2, -1).unsqueeze(0)
    cos = torch.cos(freqs).repeat_interleave(2, -1).unsqueeze(0)
    return sin, cos

def rotate_half(x):
    x_even = x[:,:,::2]
    x_odd = x[:,:,1::2]
    return torch.cat([-x_odd, x_even], dim=-1)

def rope_pos_emb(x):
    B, S, D =  x.shape
    sin, cos = get_rope_freqs(S, D)
    return cos * x + sin * rotate_half(x)

In [14]:
import torch

def get_rope_freqs(S, D):
    theta = 10000 ** (- 2 * torch.arange(0, D, 2) / D)
    pos = torch.arange(S)
    theta = torch.einsum('i,j->ij', pos, theta)
    sin = torch.sin(theta).repeat_interleave(2, -1).unsqueeze(0)
    cos = torch.cos(theta).repeat_interleave(2, -1).unsqueeze(0)
    return cos, sin

def half_rotate(x):
    x_even = x[..., ::2]
    x_odd = x[..., 1::2]
    return torch.stack([-x_odd, x_even], dim=-1).reshape(x.shape)

def apply_rotary_pos_emb(x, cos, sin):
    return cos * x + sin * half_rotate(x)

x = torch.randn(10, 256, 128)
B, S, D = x.shape
cos, sin = get_rope_freqs(S, D)
x = apply_rotary_pos_emb(x, cos, sin)